Mahin's Dataset

In [ ]:
# !wget -cq https://4yz2la.bn.files.1drv.com/y4mhrJT1X2DBj1ZXaPXCoQkkS6WwzAz_XdJDUIh_9rnt63MLJ5yy6SC23bu64NmKPripJYlHV665FGnwBxAq-1lv9VCafC0E6obLlCt_-saXYwTB13fVNiQPrUT2JZYBON1rIXRXnvnICoT7jroyS-gKBDrhxhL67FtWSC9VivIgrfn2vSeibd9ETBNypd9njplOOKRNCtG3Bpgz5Z_Ysq1YFEfCUQ5QZajU75hJp7CqDQ/nyu_data.zip

Error: Error: Failed to connect to Jupyter notebook. 
http://localhost:8888/
Error: Invalid response: 500 Internal Server Error

Yash's Dataset

In [0]:
# !wget -cp https://4iz2la.bn.files.1drv.com/y4mbkdtKoJnq_NBglCJJGvfrQdi3_-t48BOwi5p_lk_RA_R40AE2vZrXkhxo68LU6CXBI9Prxwle01FTupuJPexObohquqHgRiv1-oHAndwkJjPcoMaM3ALvTMn9go-BLYs7ehAb3tOfVCOnaHdGrtJcIQf8Zs6O82r4PzbbLJVWzsBF-Du_xtC3DutBA-ADiNHN2XZu1M8jOyRztXQxbNgbolbvmAfy9mbb_3WWN5SimM/nyu_data.zip

These two commands will download test and train dataset

In [ ]:
!wget -cq https://s3-eu-west-1.amazonaws.com/densedepth/nyu_test.zip


Error: Error: Failed to connect to Jupyter notebook. 
http://localhost:8888/
Error: Invalid response: 500 Internal Server Error

In [0]:
!wget -cq https://s3-eu-west-1.amazonaws.com/densedepth/nyu_data.zip


No Replacement for this class, its called basic policy

In [0]:
from PIL import Image, ImageEnhance, ImageOps
import numpy as np
import random

# Non-random random
random.seed(0)

class BasicPolicy(object):
    def __init__(self, mirror_ratio = 0, flip_ratio = 0, color_change_ratio = 0, is_full_set_colors = False, add_noise_peak = 0.0, erase_ratio = -1.0):
        # Random color channel order
        from itertools import product, permutations
        self.indices = list(product([0,1,2], repeat = 3)) if is_full_set_colors else list(permutations(range(3), 3))
        self.indices.insert(0, [0,1,2]) # R,G,B
        self.add_noise_peak = add_noise_peak

        # Mirror and flip
        self.color_change_ratio = color_change_ratio
        self.mirror_ratio = mirror_ratio
        self.flip_ratio = flip_ratio

        # Erase
        self.erase_ratio = erase_ratio

    def __call__(self, img, depth):

        # 0) Add poisson noise (e.g. choose peak value 20)
        # https://stackoverflow.com/questions/19289470/adding-poisson-noise-to-an-image
        if self.add_noise_peak > 0:
            PEAK = self.add_noise_peak
            img = np.random.poisson(np.clip(img, 0, 1) * PEAK) / PEAK

        # 1) Color change
        policy_idx = random.randint(0, len(self.indices) - 1)
        if random.uniform(0, 1) >= self.color_change_ratio:
            policy_idx = 0

        img = img[...,list(self.indices[policy_idx])]

        # 2) Mirror image
        if random.uniform(0, 1) <= self.mirror_ratio:
            img = img[...,::-1,:]
            depth = depth[...,::-1,:]

        # 3) Flip image vertically
        if random.uniform(0, 1) < self.flip_ratio:
            img = img[...,::-1,:,:]
            depth = depth[...,::-1,:,:]

        # 4) Erase random box
        if random.uniform(0, 1) < self.erase_ratio:
            img = self.eraser(img)

        return img, depth

    def __repr__(self):
        return "Basic Policy"

    def eraser(self, input_img, p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=True):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img
        
    def debug_img(self, img, depth, idx, i, prefix=''):
        from PIL import Image
        aug_img = Image.fromarray(np.clip(np.uint8(img*255), 0, 255))
        aug_img.save(prefix+str(idx)+"_"+str(i)+'.jpg',quality=99)
        aug_img = Image.fromarray(np.clip(np.uint8(np.tile(depth*255,3)), 0, 255))
        aug_img.save(prefix+str(idx)+"_"+str(i)+'.depth.jpg',quality=99)


This contains main data handling methods, all are needed

In [0]:
import numpy as np
#from utils import DepthNorm
from io import BytesIO
from PIL import Image
from zipfile import ZipFile
from keras.utils import Sequence
#from augment import BasicPolicy

import keras.backend as K
import tensorflow as tf

def DepthNorm(x, maxDepth):
    return maxDepth / x

def depth_loss_function(y_true, y_pred, theta=0.1, maxDepthVal=1000.0/10.0):
    
    # Point-wise depth
    l_depth = K.mean(K.abs(y_pred - y_true), axis=-1)

    # Edges
    dy_true, dx_true = tf.image.image_gradients(y_true)
    dy_pred, dx_pred = tf.image.image_gradients(y_pred)
    l_edges = K.mean(K.abs(dy_pred - dy_true) + K.abs(dx_pred - dx_true), axis=-1)

    # Structural similarity (SSIM) index
    l_ssim = K.clip((1 - tf.image.ssim(y_true, y_pred, maxDepthVal)) * 0.5, 0, 1)

    # Weights
    w1 = 1.0
    w2 = 1.0
    w3 = theta

    return (w1 * l_ssim) + (w2 * K.mean(l_edges)) + (w3 * K.mean(l_depth))
    
def extract_zip(input_zip):
    input_zip=ZipFile(input_zip)
    return {name: input_zip.read(name) for name in input_zip.namelist()}

def nyu_resize(img, resolution=480, padding=6):
    from skimage.transform import resize
    return resize(img, (resolution, int(resolution*4/3)), preserve_range=True, mode='reflect', anti_aliasing=True )

def get_nyu_data(batch_size, nyu_data_zipfile='nyu_data.zip'):
    data = extract_zip(nyu_data_zipfile)

    nyu2_train = list((row.strip().split(',') for row in (data['data/nyu2_train.csv']).decode("utf-8").split('\n') if len(row) > 0))
    nyu2_test = list((row.strip().split(',') for row in (data['data/nyu2_test.csv']).decode("utf-8").split('\n') if len(row) > 0))

    shape_rgb = (batch_size, 480, 640, 3)
    shape_depth = (batch_size, 240, 320, 1)

    return data, nyu2_train, nyu2_test, shape_rgb, shape_depth

def get_nyu_train_test_data(batch_size):
    data, nyu2_train, nyu2_test, shape_rgb, shape_depth = get_nyu_data(batch_size)

    train_generator = NYU_BasicAugmentRGBSequence(data, nyu2_train, batch_size=batch_size, shape_rgb=shape_rgb, shape_depth=shape_depth)
    test_generator = NYU_BasicRGBSequence(data, nyu2_test, batch_size=batch_size, shape_rgb=shape_rgb, shape_depth=shape_depth)

    return train_generator, test_generator

class NYU_BasicAugmentRGBSequence(Sequence):
    def __init__(self, data, dataset, batch_size, shape_rgb, shape_depth, is_flip=False, is_addnoise=False, is_erase=False):
        self.data = data
        self.dataset = dataset
        self.policy = BasicPolicy( color_change_ratio=0.50, mirror_ratio=0.50, flip_ratio=0.0 if not is_flip else 0.2, 
                                    add_noise_peak=0 if not is_addnoise else 20, erase_ratio=-1.0 if not is_erase else 0.5)
        self.batch_size = batch_size
        self.shape_rgb = shape_rgb
        self.shape_depth = shape_depth
        self.maxDepth = 1000.0

        from sklearn.utils import shuffle
        self.dataset = shuffle(self.dataset, random_state=0)

        self.N = len(self.dataset)

    def __len__(self):
        return int(np.ceil(self.N / float(self.batch_size)))

    def __getitem__(self, idx, is_apply_policy=True):
        batch_x, batch_y = np.zeros( self.shape_rgb ), np.zeros( self.shape_depth )

        # Augmentation of RGB images
        for i in range(batch_x.shape[0]):
            index = min((idx * self.batch_size) + i, self.N-1)

            sample = self.dataset[index]

            x = np.clip(np.asarray(Image.open( BytesIO(self.data[sample[0]]) )).reshape(480,640,3)/255,0,1)
            y = np.clip(np.asarray(Image.open( BytesIO(self.data[sample[1]]) )).reshape(480,640,1)/255*self.maxDepth,0,self.maxDepth)
            y = DepthNorm(y, maxDepth=self.maxDepth)

            batch_x[i] = nyu_resize(x, 480)
            batch_y[i] = nyu_resize(y, 240)

            if is_apply_policy: batch_x[i], batch_y[i] = self.policy(batch_x[i], batch_y[i])

            # DEBUG:
            #self.policy.debug_img(batch_x[i], np.clip(DepthNorm(batch_y[i])/maxDepth,0,1), idx, i)
        #exit()

        return batch_x, batch_y

class NYU_BasicRGBSequence(Sequence):
    def __init__(self, data, dataset, batch_size,shape_rgb, shape_depth):
        self.data = data
        self.dataset = dataset
        self.batch_size = batch_size
        self.N = len(self.dataset)
        self.shape_rgb = shape_rgb
        self.shape_depth = shape_depth
        self.maxDepth = 1000.0

    def __len__(self):
        return int(np.ceil(self.N / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x, batch_y = np.zeros( self.shape_rgb ), np.zeros( self.shape_depth )
        for i in range(self.batch_size):            
            index = min((idx * self.batch_size) + i, self.N-1)

            sample = self.dataset[index]

            x = np.clip(np.asarray(Image.open( BytesIO(self.data[sample[0]]))).reshape(480,640,3)/255,0,1)
            y = np.asarray(Image.open(BytesIO(self.data[sample[1]])), dtype=np.float32).reshape(480,640,1).copy().astype(float) / 10.0
            y = DepthNorm(y, maxDepth=self.maxDepth)

            batch_x[i] = nyu_resize(x, 480)
            batch_y[i] = nyu_resize(y, 240)

            # DEBUG:
            #self.policy.debug_img(batch_x[i], np.clip(DepthNorm(batch_y[i])/maxDepth,0,1), idx, i)
        #exit()

        return batch_x, batch_y

def load_test_data(test_data_zip_file='nyu_test.zip'):
    print('Loading test data...', end='')
    import numpy as np
    # from data import extract_zip
    data = extract_zip(test_data_zip_file)
    from io import BytesIO
    rgb = np.load(BytesIO(data['eigen_test_rgb.npy']))
    depth = np.load(BytesIO(data['eigen_test_depth.npy']))
    crop = np.load(BytesIO(data['eigen_test_crop.npy']))
    print('Test data loaded.\n')
    return {'rgb':rgb, 'depth':depth, 'crop':crop}

This is callback while training, again all are needed

In [0]:
import io
import random
import numpy as np
from PIL import Image

import keras
from keras import backend as K
#from utils import DepthNorm, predict, evaluate

import tensorflow as tf

def scale_up(scale, images):
    from skimage.transform import resize
    scaled = []
    
    for i in range(len(images)):
        img = images[i]
        output_shape = (scale * img.shape[0], scale * img.shape[1])
        scaled.append( resize(img, output_shape, order=1, preserve_range=True, mode='reflect', anti_aliasing=True ) )

    return np.stack(scaled)

def predict(model, images, minDepth=10, maxDepth=1000, batch_size=2):
    # Support multiple RGBs, one RGB image, even grayscale 
    if len(images.shape) < 3: images = np.stack((images,images,images), axis=2)
    if len(images.shape) < 4: images = images.reshape((1, images.shape[0], images.shape[1], images.shape[2]))
    # Compute predictions
    predictions = model.predict(images, batch_size=batch_size)
    # Put in expected range
    return np.clip(DepthNorm(predictions, maxDepth=1000), minDepth, maxDepth) / maxDepth

def evaluate(model, rgb, depth, crop, batch_size=6, verbose=True):
    # Error computaiton based on https://github.com/tinghuiz/SfMLearner
    def compute_errors(gt, pred):
        thresh = np.maximum((gt / pred), (pred / gt))
        
        a1 = (thresh < 1.25   ).mean()
        a2 = (thresh < 1.25 ** 2).mean()
        a3 = (thresh < 1.25 ** 3).mean()

        abs_rel = np.mean(np.abs(gt - pred) / gt)

        rmse = (gt - pred) ** 2
        rmse = np.sqrt(rmse.mean())

        log_10 = (np.abs(np.log10(gt)-np.log10(pred))).mean()

        return a1, a2, a3, abs_rel, rmse, log_10

    depth_scores = np.zeros((6, len(rgb))) # six metrics

    bs = batch_size

    for i in range(len(rgb)//bs):    
        x = rgb[(i)*bs:(i+1)*bs,:,:,:]
        
        # Compute results
        true_y = depth[(i)*bs:(i+1)*bs,:,:]
        pred_y = scale_up(2, predict(model, x/255, minDepth=10, maxDepth=1000, batch_size=bs)[:,:,:,0]) * 10.0
        
        # Test time augmentation: mirror image estimate
        pred_y_flip = scale_up(2, predict(model, x[...,::-1,:]/255, minDepth=10, maxDepth=1000, batch_size=bs)[:,:,:,0]) * 10.0

        # Crop based on Eigen et al. crop
        true_y = true_y[:,crop[0]:crop[1]+1, crop[2]:crop[3]+1]
        pred_y = pred_y[:,crop[0]:crop[1]+1, crop[2]:crop[3]+1]
        pred_y_flip = pred_y_flip[:,crop[0]:crop[1]+1, crop[2]:crop[3]+1]
        
        # Compute errors per image in batch
        for j in range(len(true_y)):
            errors = compute_errors(true_y[j], (0.5 * pred_y[j]) + (0.5 * np.fliplr(pred_y_flip[j])))
            
            for k in range(len(errors)):
                depth_scores[k][(i*bs)+j] = errors[k]

    e = depth_scores.mean(axis=1)

    if verbose:
        print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format('a1', 'a2', 'a3', 'rel', 'rms', 'log_10'))
        print("{:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}".format(e[0],e[1],e[2],e[3],e[4],e[5]))

    return e

def make_image(tensor):
    height, width, channel = tensor.shape
    image = Image.fromarray(tensor.astype('uint8'))
    output = io.BytesIO()
    image.save(output, format='JPEG', quality=90)
    image_string = output.getvalue()
    output.close()
    return tf.Summary.Image(height=height, width=width, colorspace=channel, encoded_image_string=image_string)

def get_nyu_callbacks(model, basemodel, train_generator, test_generator, test_set, runPath):
    callbacks = []

    # Callback: Tensorboard
    class LRTensorBoard(keras.callbacks.TensorBoard):
        def __init__(self, log_dir):
            super().__init__(log_dir=log_dir)

            self.num_samples = 6
            self.train_idx = np.random.randint(low=0, high=len(train_generator), size=10)
            self.test_idx = np.random.randint(low=0, high=len(test_generator), size=10)

        def on_epoch_end(self, epoch, logs=None):            
            if not test_set == None:
                # Samples using current model
                import matplotlib.pyplot as plt
                from skimage.transform import resize
                plasma = plt.get_cmap('plasma')

                minDepth, maxDepth = 10, 1000

                train_samples = []
                test_samples = []

                for i in range(self.num_samples):
                    x_train, y_train = train_generator.__getitem__(self.train_idx[i], False)
                    x_test, y_test = test_generator[self.test_idx[i]]

                    x_train, y_train = x_train[0], np.clip(DepthNorm(y_train[0], maxDepth=1000), minDepth, maxDepth) / maxDepth 
                    x_test, y_test = x_test[0], np.clip(DepthNorm(y_test[0], maxDepth=1000), minDepth, maxDepth) / maxDepth

                    h, w = y_train.shape[0], y_train.shape[1]

                    rgb_train = resize(x_train, (h,w), preserve_range=True, mode='reflect', anti_aliasing=True)
                    rgb_test = resize(x_test, (h,w), preserve_range=True, mode='reflect', anti_aliasing=True)

                    gt_train = plasma(y_train[:,:,0])[:,:,:3]
                    gt_test = plasma(y_test[:,:,0])[:,:,:3]

                    predict_train = plasma(predict(model, x_train, minDepth=minDepth, maxDepth=maxDepth)[0,:,:,0])[:,:,:3]
                    predict_test = plasma(predict(model, x_test, minDepth=minDepth, maxDepth=maxDepth)[0,:,:,0])[:,:,:3]

                    train_samples.append(np.vstack([rgb_train, gt_train, predict_train]))
                    test_samples.append(np.vstack([rgb_test, gt_test, predict_test]))

                self.writer.add_summary(tf.Summary(value=[tf.Summary.Value(tag='Train', image=make_image(255 * np.hstack(train_samples)))]), epoch)
                self.writer.add_summary(tf.Summary(value=[tf.Summary.Value(tag='Test', image=make_image(255 * np.hstack(test_samples)))]), epoch)
                
                # Metrics
                e = evaluate(model, test_set['rgb'], test_set['depth'], test_set['crop'], batch_size=6, verbose=True)
                logs.update({'rel': e[3]})
                logs.update({'rms': e[4]})
                logs.update({'log10': e[5]})

            super().on_epoch_end(epoch, logs)
    callbacks.append( LRTensorBoard(log_dir=runPath) )

    # Callback: Learning Rate Scheduler
    lr_schedule = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=5, min_lr=0.00009, min_delta=1e-2)
    callbacks.append( lr_schedule ) # reduce learning rate when stuck

    # Callback: save checkpoints
    callbacks.append(keras.callbacks.ModelCheckpoint(runPath + '/weights.{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss',
        verbose=1, save_best_only=False, save_weights_only=False, mode='min', period=1))

    return callbacks

A utility to perform bilinear up sampling, needed

In [0]:
from keras.engine.topology import Layer, InputSpec
import keras.utils.conv_utils as conv_utils
import tensorflow as tf
import keras.backend as K

class BilinearUpSampling2D(Layer):
    def __init__(self, size=(2, 2), data_format=None, **kwargs):
        super(BilinearUpSampling2D, self).__init__(**kwargs)
        self.data_format = K.normalize_data_format(data_format)
        self.size = conv_utils.normalize_tuple(size, 2, 'size')
        self.input_spec = InputSpec(ndim=4)

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_first':
            height = self.size[0] * input_shape[2] if input_shape[2] is not None else None
            width = self.size[1] * input_shape[3] if input_shape[3] is not None else None
            return (input_shape[0],
                    input_shape[1],
                    height,
                    width)
        elif self.data_format == 'channels_last':
            height = self.size[0] * input_shape[1] if input_shape[1] is not None else None
            width = self.size[1] * input_shape[2] if input_shape[2] is not None else None
            return (input_shape[0],
                    height,
                    width,
                    input_shape[3])

    def call(self, inputs):
        input_shape = K.shape(inputs)
        if self.data_format == 'channels_first':
            height = self.size[0] * input_shape[2] if input_shape[2] is not None else None
            width = self.size[1] * input_shape[3] if input_shape[3] is not None else None
        elif self.data_format == 'channels_last':
            height = self.size[0] * input_shape[1] if input_shape[1] is not None else None
            width = self.size[1] * input_shape[2] if input_shape[2] is not None else None
        
        return tf.image.resize_images(inputs, [height, width], method=tf.image.ResizeMethod.BILINEAR, align_corners=True)

    def get_config(self):
        config = {'size': self.size, 'data_format': self.data_format}
        base_config = super(BilinearUpSampling2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


Reduced train file and main file, change the batch size, epoch and learning rate and run this file to start the training

In [12]:
import os, sys, glob, time, pathlib

from keras import applications
from keras.models import Model, load_model
from keras.layers import Input, InputLayer, Conv2D, Activation, LeakyReLU, Concatenate
#from layers import BilinearUpSampling2D
from keras.optimizers import Adam
#from keras.utils import multi_gpu_model
#from keras.utils.vis_utils import plot_model

# Kerasa / TensorFlow
#from utilities import get_nyu_train_test_data, load_test_data, depth_loss_function
#from callbacks import get_nyu_callbacks


# Define upsampling layer
def upproject(base_model, tensor, filters, name, concat_with):
    up_i = BilinearUpSampling2D((2, 2), name=name+'_upsampling2d')(tensor)
    up_i = Concatenate(name=name+'_concat')([up_i, base_model.get_layer(concat_with).output]) # Skip connection
    up_i = Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', name=name+'_convA')(up_i)
    up_i = LeakyReLU(alpha=0.2)(up_i)
    up_i = Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', name=name+'_convB')(up_i)
    up_i = LeakyReLU(alpha=0.2)(up_i)
    return up_i

def create_model():
    
    # Encoder Layers
    print('Loading base model (DenseNet)..')
    base_model = applications.DenseNet121(input_shape=(None, None, 3), include_top=False, weights='imagenet')
    
    print('Base model loaded.')

    # Starting point for decoder
    base_model_output_shape = base_model.layers[-1].output.shape
    print(" Output shape is: ", base_model_output_shape)

    # Layer freezing?
    for layer in base_model.layers: layer.trainable = True
    
    # Starting half number of decoder filters
    
    decode_filters = int(int(base_model_output_shape[-1])/2)

    decoder = Conv2D(filters=decode_filters, kernel_size=1, padding='same', input_shape=base_model_output_shape, name='conv2')(base_model.output)
    decoder = upproject(base_model, decoder, int(decode_filters/2), 'up1', concat_with='pool3_pool')
    decoder = upproject(base_model, decoder, int(decode_filters/4), 'up2', concat_with='pool2_pool')
    decoder = upproject(base_model, decoder, int(decode_filters/8), 'up3', concat_with='pool1')
    decoder = upproject(base_model, decoder, int(decode_filters/16), 'up4', concat_with='conv1/relu')
    if False: decoder = upproject(base_model, decoder, int(decode_filters/32), 'up5', concat_with='input_1')
    
    # Extract depths (final layer)
    conv3 = Conv2D(filters=1, kernel_size=3, strides=1, padding='same', name='conv3')(decoder)
    
    # Create the model
    model = Model(inputs=base_model.input, outputs=conv3)
    print('Model created.')
    
    return (base_model,model)

def train(batch_size = 5 , epochs = 5, lr = 0.0001):
    #batch_size = int(float(args[0]))
    #epochs = int(float(args[1])) 
    #lr = float(args[2])
    print("batch_size = {0} , epochs = {1}, lr = {2}".format(batch_size,epochs, lr))

    #creates encoder and decoder model
    base_model , model = create_model()

    train_generator, test_generator = get_nyu_train_test_data( batch_size )

    # Training session details
    runPath = os.path.join(os.getcwd(),'models',str(int(time.time())))
    pathlib.Path(runPath).mkdir(parents=True, exist_ok=True)
    print('Output: ' + runPath)

    # Optimizer
    optimizer = Adam(lr=lr, amsgrad=True)

    model.compile(loss=depth_loss_function, optimizer=optimizer)

    print('Ready for training!\n')

    callbacks = []
    callbacks = get_nyu_callbacks(model, base_model, train_generator, test_generator, load_test_data(), runPath)

    # Start training
    model.fit_generator(train_generator, callbacks=None, validation_data=test_generator, epochs=epochs, shuffle=True)

    # Save the final trained model:
    print('Model Save Began')
    base_model.save(runPath + '/model.h5')
    print('Model Save Completed')
    pass

if __name__ == "__main__":
    args = sys.argv[ 1: ]
    if (len(args) <= 0) :
        sys.exit( 0 )
    
    batch_size= 5 #int(args[0]), 
    epochs= 5 #int(args[1]), 
    lr= 0.0001 #float(args[2]),
    train(batch_size,epochs,lr)

batch_size = 5 , epochs = 5, lr = 0.0001
Loading base model (DenseNet)..
Base model loaded.
 Output shape is:  (?, ?, ?, 1024)
Model created.
Output: /content/models/1575781737
Ready for training!

Loading test data...Test data loaded.

Epoch 1/5
35/35 [==============================] - 81s 2s/step - loss: 0.2463 - val_loss: 0.3804
Epoch 2/5
35/35 [==============================] - 22s 618ms/step - loss: 0.1653 - val_loss: 0.3627
Epoch 3/5
35/35 [==============================] - 22s 618ms/step - loss: 0.1443 - val_loss: 0.3601
Epoch 4/5
35/35 [==============================] - 21s 614ms/step - loss: 0.1350 - val_loss: 0.3303
Epoch 5/5
35/35 [==============================] - 22s 621ms/step - loss: 0.1261 - val_loss: 0.3398
Model Save Began
Model Save Completed
